## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
2. Pass the STAC item into the raster API `/stac/tilejson.json` endpoint.
3. Using `folium.Map`, visualize the plumes.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data

The EMIT instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with JPL Open Science and Open Data policy.

# Installing the Required Libraries
Required libraries are pre-installed on the GHG Center Hub. If you need to run this notebook elsewhere, please install them with this line in a code cell:

%pip install requests, folium, rasterstats, pystac_client, pandas, matplotlib

## Querying the STAC API
Please run the next cell to import the required libraries. 

In [33]:
import requests
import folium
import folium.plugins
from folium import Map, TileLayer 
from pystac_client import Client 
import pandas as pd
import matplotlib.pyplot as plt
import branca.colormap as cm

In [34]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://ghg.center/api/stac"
RASTER_API_URL = "https://ghg.center/api/raster"

# Please use the collection name similar to the one used in STAC collection.
# Name of the collection for methane emission plumes. 
collection_name = "emit-ch4plume-v1"

In [35]:
# Fetching the collection from STAC collections using appropriate endpoint.
# the 'requests' library allows a HTTP request possible
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection = {key: value for key, value in collection.items() if key != 'summaries'}
collection


{'id': 'emit-ch4plume-v1',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://ghg.center/api/stac/collections/emit-ch4plume-v1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://ghg.center/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://ghg.center/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://ghg.center/api/stac/collections/emit-ch4plume-v1'}],
 'title': 'Methane Point Source Plume Complexes',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-121.90662384033203,
     -39.21891784667969,
     151.0906524658203,
     50.372535705566406]]},
  'temporal': {'interval': [['2022-08-10T06:49:57+00:00',
     '2023-10-08T16:11:15+00:00']]}},
 'license': 'CC0-1.0',
 'keywords': None,
 'providers': None,
 'description': 'Methane plume complexes from point source emitters',
 'item_assets': {'ch4-plume-emissions': {'type': 'image/tiff; applicatio

Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [36]:
# Create a function that would search for the number of items in above data collection in the STAC API
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [37]:
# Apply the above function and check the total number of items available within the collection
number_of_items = get_item_count(collection_name)
plume_complexes = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(plume_complexes)} items")

Found 752 items


In [38]:
# Examine the first item in the collection
plume_complex_ids = list(map(lambda d: d.get('id', f"id not found in dictionary"), plume_complexes))
plume_complex_ids[:10]

['EMIT_L2B_CH4PLM_001_20231008T161115_001520',
 'EMIT_L2B_CH4PLM_001_20231006T100206_001584',
 'EMIT_L2B_CH4PLM_001_20231006T100206_001583',
 'EMIT_L2B_CH4PLM_001_20231006T100206_001581',
 'EMIT_L2B_CH4PLM_001_20231006T100206_001580',
 'EMIT_L2B_CH4PLM_001_20231006T100206_001579',
 'EMIT_L2B_CH4PLM_001_20231006T082735_001586',
 'EMIT_L2B_CH4PLM_001_20231006T065557_001589',
 'EMIT_L2B_CH4PLM_001_20231004T174744_001453',
 'EMIT_L2B_CH4PLM_001_20231004T174744_001452']

## Exploring Methane Emission Plumes (CH₄) using the Raster API

In this notebook, we will explore global methane emission plumes from point sources. We will visualize the outputs on a map using `folium.`

In [39]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
plume_complexes = {plume_complex["id"]: plume_complex for plume_complex in plume_complexes} 
asset_name = "ch4-plume-emissions"

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

In [40]:
# Fetching the min and max values for a specific item
rescale_values = {"max":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this for only one item so that we can visualize the event. 

In [41]:
color_map = "magma" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
plume_complex_id = "EMIT_L2B_CH4PLM_001_20230418T200118_000829"
methane_plume_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[plume_complex_id]['collection']}&item={plume_complexes[plume_complex_id]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://ghg.center/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=emit-ch4plume-v1&item=EMIT_L2B_CH4PLM_001_20230418T200118_000829&assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-638.1588745117188%2C2034.2767333984375'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-104.76285251117253,
  39.85322425220504,
  -104.74658553556483,
  39.86515336765068],
 'center': [-104.75471902336868, 39.85918880992786, 0]}

## Visualizing CH₄ Emission Plume


In [42]:
# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile["center"][1], methane_plume_tile["center"][0]), zoom_start=13)

# December 2001
map_layer = TileLayer(
    tiles=methane_plume_tile["tiles"][0],
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)

# visualising the map
map_



# Calculating Zonal Statistics
To perform zonal statistics, first we need to create a polygon. In this use case we will create a polygon around the plume.

In [43]:
# Plume AOI 
plumes_coordinates = plume_complexes[plume_complex_id]["geometry"]["coordinates"]
methane_plume_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates":
            plumes_coordinates,
        "type": "Polygon",
    },
}

In [44]:
# We'll plug in the coordinates for a location
# central to the study area and a reasonable zoom level
region_name = "Place_Holder" # please put the name of the place you are trying to visualize
aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        plumes_coordinates[0][0][1],
        plumes_coordinates[0][0][0]
    ],
    zoom_start=12,
)

folium.GeoJson(methane_plume_aoi, name=region_name).add_to(aoi_map)
aoi_map

In [45]:
colormap = "viridis" # please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html

color_map = cm.linear.viridis.scale(1,1500)
color_map.caption = 'ppm-m'
# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
plume_complex_id = "EMIT_L2B_CH4PLM_001_20230418T200118_000829"
methane_plume_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={plume_complexes[plume_complex_id]['collection']}&item={plume_complexes[plume_complex_id]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile

# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile["center"][1], methane_plume_tile["center"][0]), zoom_start=13, tiles=None)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)

# December 2001
map_layer = TileLayer(
    tiles=methane_plume_tile["tiles"][0],
    name='Plume Complex',
    overlay='True',
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)
folium.LayerControl(collapsed=False).add_to(map_)
map_.add_child(color_map)
# visualising the map
map_

## Summary


In this notebook we have successfully explored, analyzed, and visualized the STAC collection for EMIT methane emission plumes.